In [1]:
import tensorflow as tf
import numpy as np
import os
import logging
import string
import random
import yaml
from datetime import datetime
from tqdm.notebook import tqdm

from rdmtransform.model.rdmtransform import RDMtransform
from rdmtransform.model.activations import swish
from rdmtransform.training.trainer import Trainer
from rdmtransform.training.metrics import Metrics
from rdmtransform.training.data_container import DataContainer
from rdmtransform.training.data_provider import DataProvider

2023-07-14 10:01:53.255214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set up logger
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.get_logger().setLevel('WARN')
tf.autograph.set_verbosity(2)

In [3]:
with open('config.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [4]:
model_name = config['model_name']

num_basis_fct = config['num_basis_fct']
emb_size = config['emb_size']
num_blocks = config['num_blocks']

num_train = config['num_train']
num_valid = config['num_valid']
data_seed = config['data_seed']
dataset = config['dataset']
logdir = config['logdir']

num_steps = config['num_steps']
ema_decay = config['ema_decay']

learning_rate = config['learning_rate']
warmup_steps = config['warmup_steps']
decay_rate = config['decay_rate']
decay_steps = config['decay_steps']

batch_size = config['batch_size']
evaluation_interval = config['evaluation_interval']
save_interval = config['save_interval']
restart = config['restart']
comment = config['comment']
targets = config['targets']

In [5]:
data_container = DataContainer(dataset)

# Initialize DataProvider (splits dataset into training, validation and test set based on data_seed)
data_provider = DataProvider(data_container, num_train, num_valid, batch_size,
                             seed=data_seed, randomized=True)

# Initialize datasets
dataset = data_provider.get_dataset('test').prefetch(tf.data.experimental.AUTOTUNE)
dataset_iter = iter(dataset)

In [6]:
model = RDMtransform(num_basis_fct=num_basis_fct, emb_size=emb_size, num_interactions=num_blocks)

In [7]:
trainer = Trainer(model, learning_rate, warmup_steps, decay_steps, decay_rate, ema_decay, max_grad_norm=1000)

In [12]:
# Load the trained model from your own training run
directory = "models/rdmtransform/logging/20230713_182356_QWSvbRde_md_benzene2017_rdm_0_1586_E_final"  # Fill this in
best_ckpt_file = os.path.join(directory, 'best', 'best_loss.npz')
model.load_weights(best_ckpt_file)

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for models/rdmtransform/logging/20230713_182356_QWSvbRde_md_benzene2017_rdm_0_1586_E_final/best/best_loss.npz